# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Przetwarzanie-tekstu---analiza-tematów" data-toc-modified-id="Przetwarzanie-tekstu---analiza-tematów-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Przetwarzanie tekstu - analiza tematów</a></div><div class="lev2 toc-item"><a href="#Słowniczek" data-toc-modified-id="Słowniczek-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Słowniczek</a></div><div class="lev4 toc-item"><a href="#Importy" data-toc-modified-id="Importy-1101"><span class="toc-item-num">1.1.0.1&nbsp;&nbsp;</span>Importy</a></div><div class="lev3 toc-item"><a href="#Czytanie-danych" data-toc-modified-id="Czytanie-danych-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Czytanie danych</a></div><div class="lev3 toc-item"><a href="#Modelowanie-tematów" data-toc-modified-id="Modelowanie-tematów-112"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Modelowanie tematów</a></div><div class="lev4 toc-item"><a href="#Jak-to-działa?" data-toc-modified-id="Jak-to-działa?-1121"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>Jak to działa?</a></div><div class="lev3 toc-item"><a href="#LDA" data-toc-modified-id="LDA-113"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>LDA</a></div><div class="lev4 toc-item"><a href="#Jak-wybrać-liczbę-tematów?" data-toc-modified-id="Jak-wybrać-liczbę-tematów?-1131"><span class="toc-item-num">1.1.3.1&nbsp;&nbsp;</span>Jak wybrać liczbę tematów?</a></div><div class="lev3 toc-item"><a href="#Tematy-dla-artykułów" data-toc-modified-id="Tematy-dla-artykułów-114"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Tematy dla artykułów</a></div><div class="lev3 toc-item"><a href="#Wizualizacja-tematów" data-toc-modified-id="Wizualizacja-tematów-115"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Wizualizacja tematów</a></div><div class="lev3 toc-item"><a href="#Analiza-tematów-w-czasie" data-toc-modified-id="Analiza-tematów-w-czasie-116"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>Analiza tematów w czasie</a></div><div class="lev3 toc-item"><a href="#Temat-dnia" data-toc-modified-id="Temat-dnia-117"><span class="toc-item-num">1.1.7&nbsp;&nbsp;</span>Temat dnia</a></div><div class="lev2 toc-item"><a href="#Zadanie" data-toc-modified-id="Zadanie-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Zadanie</a></div>

# Przetwarzanie tekstu - analiza tematów
 W tej części warsztatów dowiesz się:
 * Jak przypisać tematy dla tekstów? - modelowanie tematów - podstawowe algorytmy i zastosowania (LDA)
 * Jak modelowanie tematów może pomóc w zrozumieniu zbioru danych tekstowych i ich wizualizacji

## Słowniczek
- LDA (*Latent Dirichlet Allocation*) - algorytm modelowania tematów z wykorzystaniem rozkładu prawdopodobieństwa Dirichleta
- standaryzacja - normalizacja zmiennej losowej, w wyniku której średnia jest równa 0, a odchylenie 1.

#### Importy

In [ ]:
# podstawowe operacje na danych
import pandas as pd
import numpy as np

# przetwarzanie tekstu, preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WordPunctTokenizer, word_tokenize

# przetwarzanie danych, modele ML
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler

# wizualizacja
import pyLDAvis
import pyLDAvis.sklearn as sklearn_vis
import plotly.plotly as py
from  plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=False)

import warnings
warnings.filterwarnings('ignore')

### Czytanie danych

Do analizy użyjemy materiałów ze zbioru danych "All the news" z Kaggle'a pochodzących z różnych serwisów internetowych.

https://www.kaggle.com/snapcrack/all-the-news/downloads/articles1.csv/4

In [ ]:
articles = pd.read_csv('articles1.csv.zip', compression='zip', index_col='id')
# analizujemy artykuły z roku 2017
articles_17 = articles[articles.year==2017]
articles_17.head()

### Modelowanie tematów

#### Jak to działa?

Tutorial na temat LDA: https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d 

Fajna prezentacja o modelowaniu tematów i LDA: http://chdoig.github.io/pytexas2015-topic-modeling

W skrócie:
- Uczenie bez nadzoru - nie wymaga wcześniejszego tagowania tekstów
- Wybieramy liczbę N tematów, które mają zostać znalezione
- Metoda operuje na słowach (ich liczbie wystąpień w dokumentach), ukrytych tematach (klastrach słów), dokumentach 
- Iteracyjny algorytm, który na początku zakłada losowy przydział tematów dla dokumentów i słów
- W kolejnych krokach aktualizuje wagi (artykuł-tematy, temat-słowa) zgodnie z obserwacjami
- Na wyjściu otrzymujemy N tematów 
  - każdy temat jest opisany przez listę powiązanych z nim słów
  - każdy dokument ma przypisane wagi (prawdopodobieństwa) powiązania z każdym z tematów

### LDA

*Latent Dirichlet Allocation* - najbardziej popularny algorytm modelowania tematów z wykorzystaniem rozkładu prawdopodobieństwa Dirichleta. Implementacje w popularnych bibliotekach (gensim, sklearn) umożliwiają wydajne douczanie modelu dla nowych tekstów.
Więcej na temat algorytmu w dokumentacji scikit-learn: http://scikit-learn.org/stable/modules/decomposition.html#latentdirichletallocation


In [ ]:
def preprocess_tokens(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.tokenize.regexp_tokenize(text, '[a-zA-Z]{3,}')
    return [lemmatizer.lemmatize(word).lower() for word in tokens]


def fit_lda(txt, n_topics):
    vectorizer = CountVectorizer(stop_words='english', tokenizer=preprocess_tokens, max_df=0.5, min_df=10)
    lda = LatentDirichletAllocation(n_topics=n_topics, verbose=1)
    doc_tf = vectorizer.fit_transform(txt)
    lda_mdl = lda.fit(doc_tf)
    topic_words = pd.DataFrame(lda.components_).apply(lambda topic:
                                       ' '.join([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-5 - 1:-1]]), 1)
    print(topic_words)
    return vectorizer, lda_mdl, topic_words


def transform_lda(txt, vectorizer, lda):
    docs_tf = vectorizer.transform(txt)
    doc_topics = lda.transform(docs_tf)
    return docs_tf, doc_topics

Należy zdefiniować liczbę tematów, które zostaną znalezione.

In [ ]:
n_topics = 50
vectorizer, lda, topic_words = fit_lda(articles_17.content, n_topics)

#### Jak wybrać liczbę tematów?
Im większa liczba, tym bardziej szczegółowe tematy otrzymamy.

Do wyboru liczby tematów często używa się miary "perplexity" do sprawdzenia, czy model dobrze odwzorowuje rozkład słów w próbce.
Jednak z badań wynika, że ta miara często nie koreluje z ludzką ewaluacją jakości modelu - http://qpleple.com/perplexity-to-evaluate-topic-models/

Dlatego często najlepiej najpierw "ręcznie" zapoznać się ze zwróconymi tematami i ocenić ich jakość. 

### Tematy dla artykułów

Do każdego tekstu przypisujemy temat z największą wagą.

In [ ]:
docs_tf, doc_topics = transform_lda(articles.content, vectorizer, lda)

In [ ]:
max_topics = np.argmax(doc_topics, 1)
for i in range(10,20):
    print("Text: ")
    print(articles.content.iloc[i][:200])
    print("Topic: ")
    print(topic_words.iloc[max_topics[i]])
    print()
    

### Wizualizacja tematów

Dla lepszego zrozumienia i analizy otrzymanego modelu warto zwizualizować otrzymane grupy tematów. 
Bardzo pomocny jest pakiet LDAvis, który tworzy wykresy dla modeli tematów (kompatybilny z formatem sklearn, gensim).

In [ ]:
topics_vis = sklearn_vis.prepare(lda, docs_tf, vectorizer)
pyLDAvis.display(topics_vis)

### Analiza tematów w czasie

Możemy wykorzystać modelowanie tematów do znalezienia "tematów dnia" w danym okresie.




### Temat dnia

Szukamy tematów, które danego dnia miały wyższe średnie wartości, niż przez pozostały okres - tzn. więcej artykułów związancyh z nimi było tego dnia opublikowanych.

In [ ]:
scaler = StandardScaler()
docs_tf, doc_topics = transform_lda(articles_17.content, vectorizer, lda)

In [ ]:
articles_topics = pd.DataFrame(scaler.fit_transform(doc_topics), index=articles_17.index, columns=topic_words)
mean_topics = articles_topics.merge(articles_17, left_index=True, right_on='id').groupby('date').mean()[topic_words]

In [ ]:
traces = [go.Scatter(
    x = mean_topics.index,
    y = mean_topics[topic_words[i]],
    text = topic_words[i],
    name = topic_words[i]
) for i in range(n_topics)]


iplot(traces, filename='topic_time')

## Zadanie

Model tematów - opisy TED-ów

Zbuduj model tematów dla danych z https://www.kaggle.com/goweiting/ted-talks-transcript#ted_metadata_kaggle.csv

1. Zbuduj model tematów (N=10/20)
2. Stwórz wizualizację dla otrzymanego modelu (za pomocą LDA-vis)